# Agent : Learning

## ML Strategy

# -------------------------------------------------

## Backtest

In [1]:

from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2022"
END = "2024"
TICKERS = ["BTC", "ETH", "SOL", "QNT", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

SYMBOL = "QNT"
INTERVAL = "1d"
CAPITAL = 1000


config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)

sub_env = Env(symbol=SYMBOL, interval=INTERVAL, start=START, end=END)

# Agent details
CAPITAL = 1000
LEVERAGE = 5

n_SESSION = 100

AGENT_PARAMS = [
    [(1, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(2, "BTC"), CAPITAL, sub_env, n_SESSION],
    [(3, "BTC"), CAPITAL, sub_env, n_SESSION]
]


STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4}),
    ("MOMENTUM", 8, {"floor" : 4}),
    ("TRIPLEMA", (14, 21, 35), {"floor" : 4})
]

STRATEGIES = [
    ("TRIPLEMA", (3, 7, 14), {"floor" : 4})
]


STRATEGIES = [
    ("MOMENTUM", 3, {"floor" : 4}),
    ("MOMENTUM", 5, {"floor" : 4}),
    ("MOMENTUM", 6, {"floor" : 4})
]

AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    


[(1, 'BTC'), 1000, <system.env.Env object at 0x0000024FEEC63F50>, 100]
{'date': Timestamp('2022-04-09 00:00:00'), 'nbTrades': 10, 'winRate': 0.3, 'lossRate': -0.7, 'amountWin': 133.4628879955868, 'amountLoss': -234.97453700023686, 'totalAmount': -101.51164900465005, 'amount': -92.84321435036986, 'expectancy': -10.151164900465002, 'profitFactor': 0.5679887263506015, 'minExposure': 2, 'maxExposure': 2, 'q1 avg median q3 %': [-7.0868, -0.9722, -7.039, -6.9913]}
__ End Session 0 ____________
__ Start Session 1____________
{'date': Timestamp('2022-07-19 00:00:00'), 'nbTrades': 21, 'winRate': 0.3333333333333333, 'lossRate': -0.6666666666666666, 'amountWin': 531.6554154213645, 'amountLoss': -413.5127165921142, 'totalAmount': 118.14269882925032, 'amount': 141.89556779282407, 'expectancy': 5.625842801392871, 'profitFactor': 1.2857051164058526, 'minExposure': 2, 'maxExposure': 2, 'q1 avg median q3 %': [-8.3204, 0.8891, -8.2985, -8.2766]}
__ End Session 1 ____________
__ Start Session 2__________

In [2]:
AGENT[0].view_report()

(1, 'BTC')


In [3]:
AGENT[0].session.rets_dist.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7])

In [4]:
AGENT[0].get_report()

Initial Capital: 1000
Agent Capital: 998.7120898424372
Portfolio Capital: 887.6579281078374
PnL: -1.2879101575628056
Return :  -0.12879101575627994 %
Min exposure : 2  Max exposure : 2


,date,nbTrades,winRate,lossRate,profitFactor
0,2022-04-09,10,0.300000,-0.700000,0.567989
1,2022-07-19,21,0.333333,-0.666667,1.285705
2,2022-10-28,21,0.285714,-0.714286,0.878575
3,2023-02-06,21,0.238095,-0.761905,0.906733
4,2023-05-18,20,0.400000,-0.600000,2.019551
5,2023-08-27,18,0.277778,-0.722222,1.648123
6,2023-12-06,20,0.400000,-0.600000,0.861350
7,2024-03-16,21,0.476190,-0.523810,2.154103
8,2022-04-09,9,0.444444,-0.555556,0.299524
9,2022-07-19,18,0.222222,-0.777778,0.704385


In [ ]:
session = AGENT[0].post_event.sessionData
session

# -------------------------------------------------

In [ ]:
from system.agent import Agent
from system.env import Env, GEnv

from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


START = "2021"
END = "2024"
TICKERS = ["BTC", "ETH", "SOL", "QNT", "DASH", "MATIC", "TWT", "GALA", "EGLD", "XMR"]

SYMBOL = "QNT"
INTERVAL = "1d"
CAPITAL = 1000


config = {
    "symbols" : TICKERS, "interval" : INTERVAL, "start" : START, "end" : END
    }
env = GEnv(**config)

sub_env = Env(symbol=SYMBOL, interval=INTERVAL, start=START, end=END)

# Agent details
CAPITAL = 1000
LEVERAGE = 5

AGENT_PARAMS = [
    [(1, SYMBOL), CAPITAL, sub_env, n_SESSION]
]


STRATEGIES = [
    ("MOMENTUM", 6, {"floor" : 4})
]


AGENT = []
for agent_x, strategie in zip(AGENT_PARAMS, STRATEGIES):
    print(agent_x)
    agent = Agent(*agent_x)
    agent.update_policy(*strategie)
    agent.run_episode()
    AGENT.append(agent)
    


In [ ]:
AGENT[0].view_report()

In [ ]:
pp = AGENT[0].env.post_event.portfolioData
pp.head()

In [ ]:
dd = AGENT[0].env.post_event.tradeData
dd

In [ ]:
import ast

serie = dd["pnl_pct"].values
serie = dd[dd["state"].apply(lambda x : ast.literal_eval(x)[0]) == 'Close']["pnl_pct"].values

In [ ]:
import ast

dd["state"].apply(lambda x : ast.literal_eval(x)[1]).iloc[-1]

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Boxplots
fig_box = go.Figure()
fig_box.add_trace(go.Box(y=serie_pos, name='Positives', marker_color='blue'))
fig_box.add_trace(go.Box(y=serie_neg, name='Négatives', marker_color='red'))
fig_box.update_layout(title_text='Boxplots des valeurs positives et négatives', yaxis_title='Valeur')

fig_box.update_layout(height = 800 , width = 1000,
                          legend = dict(orientation="h",
                                        yanchor="bottom", y=1,
                                        xanchor="right", x=0.5),
                          margin = {'t':0, 'b':0, 'l':10, 'r':0}
                          )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Exemple de série de données
data = serie  # Génère 1000 valeurs aléatoires selon une distribution normale

# Séparer les données en positives et négatives
positive_data = data[data > 0]
negative_data = data[data < 0]

# Afficher les histogrammes
plt.figure(figsize=(12, 6))

# Histogramme des valeurs positives
plt.subplot(1, 2, 1)
plt.hist(positive_data, bins=30, color='blue', alpha=0.7, edgecolor='black')
plt.title('Histogramme des valeurs positives')
plt.xlabel('Valeur')
plt.ylabel('Fréquence')

# Histogramme des valeurs négatives
plt.subplot(1, 2, 2)
plt.hist(negative_data, bins=30, color='red', alpha=0.7, edgecolor='black')
plt.title('Histogramme des valeurs négatives')
plt.xlabel('Valeur')
plt.ylabel('Fréquence')

plt.tight_layout()
plt.show()

# Afficher les KDE
plt.figure(figsize=(12, 6))

# KDE des valeurs positives
sns.kdeplot(positive_data, shade=True, color='blue', label='Positives')
# KDE des valeurs négatives
sns.kdeplot(negative_data, shade=True, color='red', label='Négatives')

plt.title('Estimation de la densité de probabilité')
plt.xlabel('Valeur')
plt.ylabel('Densité')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = serie

# Séparer les données en positives et négatives
positive_data = data[data > 0]
negative_data = data[data < 0]

# Création de la figure et des sous-graphiques
fig, axs = plt.subplots(2, 2, figsize=(14, 12))

# Histogramme et KDE des valeurs positives
sns.histplot(positive_data, bins=30, kde=True, color='blue', ax=axs[0, 0])
axs[0, 0].set_title('Histogramme et KDE des valeurs positives')
axs[0, 0].set_xlabel('Valeur')
axs[0, 0].set_ylabel('Fréquence/Densité')

# Histogramme et KDE des valeurs négatives
sns.histplot(negative_data, bins=30, kde=True, color='red', ax=axs[0, 1])
axs[0, 1].set_title('Histogramme et KDE des valeurs négatives')
axs[0, 1].set_xlabel('Valeur')
axs[0, 1].set_ylabel('Fréquence/Densité')

# Boxplot des valeurs positives
sns.boxplot(data=positive_data, ax=axs[1, 0], color='blue')
axs[1, 0].set_title('Boxplot des valeurs positives')
axs[1, 0].set_xlabel('Valeur')

# Boxplot des valeurs négatives
sns.boxplot(data=negative_data, ax=axs[1, 1], color='red')
axs[1, 1].set_title('Boxplot des valeurs négatives')
axs[1, 1].set_xlabel('Valeur')

plt.tight_layout()
plt.show()


In [ ]:
port = AGENT[0].env.post_event.portfolioData
port

#### --------------------------------------

# *Multi-Agent* v2(1)

In [ ]:
from magent import MAgentThread
from master import  MasterAgentThread
from system.env import Env, SubEnv

from threading import Thread, Condition, Event, Barrier, Semaphore

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2


TICKERS = ["BTC", "ETH", "SOL", "DASH"]
START = "2022-01-01"
END = "2023-10-01"
INTERVAL = "1d"
CAPITAL = 100

config = {"capital" : CAPITAL, "interval" : INTERVAL,
          "symbols" : TICKERS, "start" : START, "end" : END}
env = Env(**config)

sub_env_1 = SubEnv(symbol="BTC", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_2 = SubEnv(symbol="ETH", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_3 = SubEnv(symbol="SOL", capital=CAPITAL, interval=INTERVAL, start=START, end=END)
sub_env_4 = SubEnv(symbol="EGLD", capital=CAPITAL, interval=INTERVAL, start=START, end=END)

# Agent details
AGENT_PARAMS = [
    [(1, "BTC"), sub_env_1],
    [(2, "ETH"), sub_env_2],
    [(3, "SOL"), sub_env_3],
    [(4, "EGLD"), sub_env_4]
]

N = len(AGENT_PARAMS)

STRATEGIES = [
    ((1, "BTC"), "MOMENTUM", 3),
    ((2, "ETH"), "TRIPLEMA", (3, 7, 14)),
    ((3, "SOL"), "MOMENTUM", 8),
    ((4, "EGLD"), "TRIPLEMA", (14, 21, 35))
]

In [ ]:
master = MasterAgentThread(env)
master.set_barrier(n=N)

for agent_param, strategie in zip(AGENT_PARAMS, STRATEGIES):
    master.add_agent(*agent_param)
    master.update_params(*strategie)


master.active_agents()
master.run()

In [ ]:
master.agents

In [ ]:
#master.agents[(1, "BTC")].env.get_viz(1, "BTC")
master.agents[(AGENT_PARAMS[0][0])].env.get_viz(*AGENT_PARAMS[0][0])

In [ ]:
master.agents[(AGENT_PARAMS[1][0])].env.get_viz(*AGENT_PARAMS[1][0])

In [ ]:
master.agents[(AGENT_PARAMS[2][0])].env.get_viz(*AGENT_PARAMS[2][0])

In [ ]:
master.agents[(AGENT_PARAMS[3][0])].env.get_viz(*AGENT_PARAMS[3][0])

In [ ]:
master.agents[(4, "EGLD")].env.journal.portfolioData
#master.agents[(3, "SOL")].env.journal.tradesData

In [ ]:
master.agents

In [ ]:
master.agents[(3, "SOL")].env.journal.portfolioData

# *End*

In [ ]:
import threading

threading.enumerate()

# *Multi-Agent v3* Multiprocessing

In [ ]:
import multiprocessing
from multiprocessing import Process, Manager, Queue, Barrier, Event, Semaphore



# Plot

In [ ]:
import plotly.graph_objects as go

categories = ['processing cost','mechanical properties','chemical stability',
              'thermal stability', 'device integration']

fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=[1, 5, 2, 2, 3],
      theta=categories,
      fill='toself',
      name='Product A'
))
fig.add_trace(go.Scatterpolar(
      r=[4, 3, 2.5, 1, 2],
      theta=categories,
      fill='toself',
      name='Product B'
))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 5]
    )),
  showlegend=False
)

fig.show()

In [ ]:
class MyGenerator:
    def __init__(self, n):
        self.n = n
        self.current = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.current < self.n:
            result = self.current
            self.current += 1
            return result
        else:
            raise StopIteration

# Créez une instance de la classe MyGenerator
dix_nombres = MyGenerator(10)

# Utilisez une boucle for pour obtenir les valeurs
for nombre in dix_nombres:
    print(nombre)


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
import os 

os.cpu_count()

In [ ]:
import threading

class Master:
    def __init__(self, condition, event):
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print("Maître : J'attends que les agents soient prêts.")
                self.condition.wait()
                print("Maître : Autorisation d'exécuter le travail.")
                self.event.set()
                self.condition.notify_all()

class Agent:
    def __init__(self, agent_id, condition, event):
        self.agent_id = agent_id
        self.condition = condition
        self.event = event

    def run(self):
        for _ in range(10):
            with self.condition:
                print(f"Agent {self.agent_id} : Prêt à travailler.")
                self.condition.notify()
                self.condition.wait()
                self.event.wait()
                self.event.clear()
                print(f"Agent {self.agent_id} : Travail en cours.")
                self.condition.notify()
                self.condition.wait()
                print(f"Agent {self.agent_id} : Travail terminé.")

# Créez une Condition pour coordonner les agents et le maître
condition = threading.Condition()

# Créez un Event pour autoriser les agents à travailler
event = threading.Event()

# Créez le maître et les agents
master = Master(condition, event)
agents = [Agent(i, condition, event) for i in range(1, 4)]

# Créez des threads pour le maître et les agents
master_thread = threading.Thread(target=master.run)
agent_threads = [threading.Thread(target=agent.run) for agent in agents]

# Démarrez les threads
master_thread.start()
for thread in agent_threads:
    thread.start()

# Attendez que tous les threads se terminent
master_thread.join()
for thread in agent_threads:
    thread.join()

print("Tous les agents et le maître ont terminé.")


In [ ]:
import time
from threading import Thread
from queue import Queue

class Agent:
    def __init__(self, master_queue, name):
        self.master_queue = master_queue
        self.name = name

    def do_task(self, task):
        print(f"Agent {self.name} received task: {task}")
        # Simulating task execution
        time.sleep(2)
        print(f"Agent {self.name} completed task: {task}")
        # Inform the master that the task is completed
        self.master_queue.put((self.name, task))


class Master:
    def __init__(self):
        self.agent_queues = {}

    def add_agent(self, agent):
        agent_queue = Queue()
        self.agent_queues[agent.name] = agent_queue
        Thread(target=self.listen_to_agent, args=(agent, agent_queue)).start()

    def assign_task(self, agent_name, task):
        agent_queue = self.agent_queues.get(agent_name)
        if agent_queue:
            print(f"Master assigned task: {task} to Agent {agent_name}")
            agent_queue.put(task)
        else:
            print(f"Agent {agent_name} not found.")

    def listen_to_agent(self, agent, agent_queue):
        while True:
            task = agent_queue.get()
            if task is None:
                break
            print(f"Master received task completion report from Agent {agent.name} for task: {task}")


# Exemple d'utilisation
if __name__ == "__main__":
    master = Master()

    agents = [Agent(master_queue=master, name=f"Agent{i+1}") for i in range(4)]

    for agent in agents:
        master.add_agent(agent)

    master.assign_task("Agent1", "Task1")
    master.assign_task("Agent2", "Task2")
    master.assign_task("Agent3", "Task3")
    master.assign_task("Agent4", "Task4")

    # Attendez la fin des tâches avant de quitter
    time.sleep(3)  # Simuler le temps nécessaire pour terminer les tâches
    for agent in agents:
        master.add_agent(agent)  # Envoyer un signal de fin à tous les agents


In [ ]:
import multiprocessing
import time
import random

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_queue, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_queue = master_queue
        self.agent_queue = agent_queue

    def run(self):
        while True:
            instruction = self.master_queue.get()

            if instruction == "QUIT":
                break

            # Ajoutez des conditions en fonction de l'instruction
            if instruction == "Commande 1":
                result = f"Agent {self.agent_id} exécute la Commande 1 spécifique."
            elif instruction == "Commande 2":
                result = f"Agent {self.agent_id} exécute la Commande 2 spécifique."
            else:
                result = f"Agent {self.agent_id} exécute une instruction non gérée."

            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_queue = multiprocessing.Queue()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_queue, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            for agent_queue in self.agent_queues:
                self.master_queue.put(instruction)

            time.sleep(1)  # Attendez un certain temps pour simuler le traitement parallèle

            for _ in range(4):
                report = self.master_queue.get()
                print(report)

    def stop_agents(self):
        for _ in range(4):
            self.master_queue.put("QUIT")

        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:
import multiprocessing
import time

class Agent(multiprocessing.Process):
    def __init__(self, agent_id, master_event, agent_queue):
        super().__init__()
        self.agent_id = agent_id
        self.master_event = master_event
        self.agent_queue = agent_queue

    def run(self):
        while True:
            # Wait for the event to be set by the master
            self.master_event.wait()

            instruction = self.agent_queue.get()

            if instruction == "QUIT":
                break

            # Your conditions or logic here
            result = f"Agent {self.agent_id} a exécuté l'instruction: {instruction}"

            # Report the result to the master
            self.agent_queue.put(result)

class Master:
    def __init__(self):
        self.master_event = multiprocessing.Event()
        self.agent_queues = [multiprocessing.Queue() for _ in range(4)]
        self.agents = [Agent(i, self.master_event, self.agent_queues[i]) for i in range(4)]

    def start_agents(self):
        for agent in self.agents:
            agent.start()

    def send_instructions(self, instructions):
        for instruction in instructions:
            # Send the instruction to all agents
            for agent_queue in self.agent_queues:
                agent_queue.put(instruction)

            # Set the event to signal agents to start processing
            self.master_event.set()

            time.sleep(1)  # Simulate parallel processing time

            # Clear the event for the next round
            self.master_event.clear()

            # Collect reports from agents
            for _ in range(4):
                report = self.agent_queues[0].get()  # Assuming all agents provide the same result
                print(report)

    def stop_agents(self):
        # Send the QUIT instruction to all agents
        for agent_queue in self.agent_queues:
            agent_queue.put("QUIT")

        # Wait for agents to finish
        for agent in self.agents:
            agent.join()

if __name__ == "__main__":
    master_instance = Master()
    master_instance.start_agents()

    instructions = ["Commande 1", "Commande 2", "Commande 3", "Commande 4"]
    master_instance.send_instructions(instructions)

    master_instance.stop_agents()


In [ ]:


class ProfitManagement:
    def __init__(self, profit_target_percentage, trailing_stop_percentage):
        self.profit_target_percentage = profit_target_percentage
        self.trailing_stop_percentage = trailing_stop_percentage
        self.entry_price = None
        self.profit_target_price = None
        self.trailing_stop_price = None

    def set_entry_price(self, entry_price):
        self.entry_price = entry_price
        self.profit_target_price = entry_price * (1 + self.profit_target_percentage)
        self.trailing_stop_price = entry_price * (1 - self.trailing_stop_percentage)

    def take_profit(self, current_price):
        if current_price >= self.profit_target_price:
            return True
        return False

    def trailing_stop_loss(self, current_price):
        if current_price >= self.trailing_stop_price:
            self.trailing_stop_price = current_price * (1 - self.trailing_stop_percentage)
        return self.trailing_stop_price

# Exemple d'utilisation
profit_target_percentage = 0.05  # Objectif de profit de 5%
trailing_stop_percentage = 0.02  # Trailing stop-loss de 2%

profit_manager = ProfitManagement(profit_target_percentage, trailing_stop_percentage)
profit_manager.set_entry_price(100)  # Prix d'entrée de la position

# Simulation du mouvement des prix
price_history = [100, 110, 120, 115, 125, 130, 135, 130, 140, 145]

for price in price_history:
    if profit_manager.take_profit(price):
        print(f"Prendre profit au prix {price}")
        break
    else:
        trailing_stop_price = profit_manager.trailing_stop_loss(price)
        print(f"Prix actuel : {price}, Trailing stop-loss : {trailing_stop_price}")


In [2]:
#import dash_table
from dash import dash_table

In [ ]:
# ... autres importations ...
import dash_table

def create_layout():
    return dbc.Container([
        create_navbar(),
        dbc.Row([
            dbc.Col(html.H1("Binance Futures Trading Dashboard", className="text-center mb-4 text-white"), width=12)
        ], className="mt-4"),
        
        # Wallet Overviews Section
        dbc.Row([
            dbc.Col([
                html.H3([
                    html.I(className="fas fa-chart-line me-2"),
                    "Wallet Overviews"
                ], className="text-white"),
                dcc.Interval(id='interval-component', interval=10*1000, n_intervals=0),
                dcc.Graph(id='waterfall-graph', config={'displayModeBar': False}),
            ], width=6),
            dbc.Col(html.Div(id='risk-indicators'), width=6)
        ], className="mt-4"),
        
        # Positions Actuelles Section
        dbc.Row([
            dbc.Col([
                html.H3([
                    html.I(className="fas fa-chart-line me-2"),
                    "Positions Actuelles"
                ], className="text-white"),
                dash_table.DataTable(
                    id='positions-table',
                    columns=[
                        {"name": "Symbol", "id": "symbol"},
                        {"name": "Position Amt", "id": "positionAmt"},
                        {"name": "Entry Price", "id": "entryPrice"},
                        {"name": "Current Price", "id": "currentPrice"},
                        {"name": "Unrealized Profit", "id": "unrealizedProfit"},
                        {"name": "Leverage", "id": "leverage"},
                        {"name": "Status", "id": "status"},
                        {"name": "Close Position", "id": "close_position", "presentation": "button"}
                    ],
                    data=[],
                    editable=False,
                    row_deletable=False,
                    style_cell={'textAlign': 'center', 'backgroundColor': 'black', 'color': 'white'},
                    style_header={'backgroundColor': 'grey'},
                    style_data_conditional=[
                        {
                            'if': {'column_id': 'unrealizedProfit', 'filter_query': '{unrealizedProfit} > 0'},
                            'backgroundColor': 'green',
                            'color': 'white'
                        },
                        {
                            'if': {'column_id': 'unrealizedProfit', 'filter_query': '{unrealizedProfit} < 0'},
                            'backgroundColor': 'red',
                            'color': 'white'
                        }
                    ]
                )
            ], width=12)
        ], className="mt-4"),
        
        # OHLC and Order Placement Section
        dbc.Row([
            dbc.Col([
                html.H3([
                    html.I(className="fas fa-wallet me-2"),
                    "OHLC"
                ], className="text-white"),
                dcc.Graph(id='ohlc-graph', config={'displayModeBar': False})
            ], width=10),  # 70% width
            dbc.Col([
                html.H3([
                    html.I(className="fas fa-hand-holding-usd me-2"),
                    "Placer un Ordre"
                ], className="text-white"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Symbole", className="text-white"),
                        dcc.Dropdown(SYMBOLS, id='order-symbol', value='BTCUSDT', className="bg-dark text-white")
                    ], width=12)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Type d'ordre", className="text-white"),
                        dcc.RadioItems(
                            id='order-type',
                            options=[
                                {'label': 'Market', 'value': 'MARKET'},
                                {'label': 'Limit', 'value': 'LIMIT'}
                            ],
                            value='MARKET',
                            className="bg-dark text-white")
                    ], width=12)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Quantité/Unit", className="text-white"),
                        dcc.Input(id='order-quantity', type='number', className="bg-dark text-white")
                    ], width=12)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Limit Price", className="text-white"),
                        dcc.Input(id='order-price', type='number', className="bg-dark text-white")
                    ], width=12)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Cours Actuel", className="text-white"),
                        html.Div(id='current-price', className="text-white mt-2")
                    ], width=12)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Stop Loss", className="text-white"),
                        dcc.Input(id='order-SL', type='number', className="bg-dark text-white")
                    ], width=6),
                    dbc.Col([
                        dbc.Label("Take Profit", className="text-white"),
                        dcc.Input(id='order-TP', type='number', className="bg-dark text-white")
                    ], width=6)
                ], className="mb-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Label("Levier", className="text-white"),
                        dcc.Slider(1, 20, 4, id='order-leverage', className='bg-dark text-white')
                    ], width=12)
                ], className="mb-2"),
                html.Div(id='order-preview', className="text-white mt-2"),
                dbc.Row([
                    dbc.Col([
                        dbc.Button("Long", id='order-long', color="success", n_clicks=0, className="me-2")
                    ], width=6),
                    dbc.Col([
                        dbc.Button("Short", id='order-short', color="danger", n_clicks=0, className="me-2")
                    ], width=6)
                ]),
                html.Div(id='order-result', className="text-white mt-2")
            ], width=2)  # 30% width
        ], className="mt-4"),

        # Events Section
        dbc.Row([
            dbc.Col([
                html.H3([
                    html.I(className="fas fa-list-alt me-2"),
                    "Événements"
                ], className="text-white"),
                html.Div(id='events-div', className="text-white")
            ], width=12)
        ], className="mt-4")
    ], fluid=True)

# Create layout function to be used in app
layout = create_layout()


# Tableau de Bord de Trading Binance Futures

Cette application est un tableau de bord de trading pour Binance Futures, offrant diverses fonctionnalités pour gérer et suivre vos positions de trading.

## Fonctionnalités Principales

### Visualisation des Positions Actuelles
- Affiche les positions ouvertes en temps réel.
- Met à jour les données des positions toutes les 10 secondes.

### Statistiques du PnL
- Affiche les statistiques du profit et perte (PnL) en temps réel.
- Met à jour les données du PnL toutes les 10 secondes.

### Historique des Positions
- Permet de sélectionner une période spécifique pour afficher l'historique des ordres.
- Affiche un tableau des ordres passés et un graphique des prix au fil du temps pour les symboles sélectionnés.
- Affiche des points d'entrée et de sortie sur le graphique des prix si des positions existent pour le symbole sélectionné.

### Graphiques Détaillés
- Visualise les transactions individuelles avec des points d'entrée et de sortie sur les graphiques de prix.
- Affiche un graphique détaillé des transactions pour les symboles sélectionnés.

### Actifs Détenus
- Affiche les actifs actuellement détenus, limités à trois actifs au maximum.
- Met à jour les graphiques des actifs détenus toutes les 10 secondes.

### Placement d'Ordres
- Permet de placer des ordres de type Market et Limit.
- Sélection du symbole de trading parmi toutes les paires disponibles sur Binance.
- Sélection du côté de l'ordre (Long/Short) et du type d'ordre (Market/Limit).
- Prévisualisation du total de l'ordre avant de le placer.
- Affichage du résultat de l'ordre après le placement (succès ou échec).

### Journal des Événements
- Affiche un journal des événements de trading, y compris les ordres placés, réussis ou échoués.
- Met à jour les événements toutes les 10 secondes.

### Évolution du Capital
- Affiche un graphique de l'évolution du capital en temps réel.
- Met à jour les données du capital toutes les 10 secondes.

## Technologies Utilisées
- **Dash** : Pour la visualisation des données et les composants interactifs.
- **Dash Bootstrap Components** : Pour le style et les composants visuels.
- **API Binance** : Pour récupérer les données de trading.

## Fonctionnalités Interactives
- **Sélection de la Période** : Permet de sélectionner une période spécifique pour afficher l'historique des positions.
- **Sélection de la Paire de Trading** : Permet de sélectionner une paire de trading parmi toutes les paires disponibles sur Binance.
- **Points d'Entrée et de Sortie** : Affiche les points d'entrée et de sortie sur les graphiques de prix pour les positions existantes.
- **Placement d'Ordres** : Permet de placer des ordres de trading avec prévisualisation du total de l'ordre.
- **Graphiques Dynamiques** : Affiche des graphiques détaillés des transactions et de l'évolution du capital.

## Mise à Jour en Temps Réel
- **Positions Actuelles** : Mise à jour toutes les 10 secondes.
- **Statistiques du PnL** : Mise à jour toutes les 10 secondes.
- **Actifs Détenus** : Mise à jour toutes les 10 secondes.
- **Journal des Événements** : Mise à jour toutes les 10 secondes.
- **Évolution du Capital** : Mise à jour toutes les 10 secondes.
